# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import logictensornetworks as ltn
import tensorflow as tf
import time

# Dataset exploration

In [2]:
start_time = time.time()
df = pd.read_csv("kddcup99.csv", skipinitialspace = True)

In [3]:
df.shape #numar de linii si coloane

(494020, 42)

In [4]:
df.drop_duplicates(keep= 'first', inplace=True) #eliminarea dublurilor liniilor
df.shape

(145585, 42)

In [5]:
df.describe()

duration     src_bytes     dst_bytes           land  \
count  145585.000000  1.455850e+05  1.455850e+05  145585.000000   
mean      132.026088  7.995752e+03  2.859788e+03       0.000137   
std      1224.161209  1.820390e+06  6.081000e+04       0.011720   
min         0.000000  0.000000e+00  0.000000e+00       0.000000   
25%         0.000000  0.000000e+00  0.000000e+00       0.000000   
50%         0.000000  1.470000e+02  1.050000e+02       0.000000   
75%         0.000000  2.880000e+02  1.164000e+03       0.000000   
max     58329.000000  6.933756e+08  5.155468e+06       1.000000   

       wrong_fragment         urgent            hot  num_failed_logins  \
count   145585.000000  145585.000000  145585.000000      145585.000000   
mean         0.020201       0.000048       0.100175           0.000515   
std          0.239369       0.010150       1.426803           0.028586   
min          0.000000       0.000000       0.000000           0.000000   
25%          0.000000       0.000000       0.000000           0.000000   
50%          0.000000       0.000000       0.000000           0.000000   
75%          0.000000       0.000000       0.000000           0.000000   
max          3.000000       3.000000      30.000000           5.000000   

           logged_in  lnum_compromised  ...  dst_host_count  \
count  145585.000000     145585.000000  ...   145585.000000   
mean        0.491486          0.026177  ...      181.469423   
std         0.499929          3.311384  ...       99.097396   
min         0.000000          0.000000  ...        0.000000   
25%         0.000000          0.000000  ...       78.000000   
50%         0.000000          0.000000  ...      255.000000   
75%         1.000000          0.000000  ...      255.000000   
max         1.000000        884.000000  ...      255.000000   

       dst_host_srv_count  dst_host_same_srv_rate  dst_host_diff_srv_rate  \
count       145585.000000           145585.000000           145585.000000   
mean           129.935570                0.553226                0.061389   
std            114.714717                0.456236                0.147021   
min              0.000000                0.000000                0.000000   
25%             12.000000                0.050000                0.000000   
50%            117.000000                0.770000                0.020000   
75%            255.000000                1.000000                0.070000   
max            255.000000                1.000000                1.000000   

       dst_host_same_src_port_rate  dst_host_srv_diff_host_rate  \
count                145585.000000                145585.000000   
mean                      0.092826                     0.019048   
std                       0.241124                     0.060410   
min                       0.000000                     0.000000   
25%                       0.000000                     0.000000   
50%                       0.000000                     0.000000   
75%                       0.030000                     0.020000   
max                       1.000000                     1.000000   

       dst_host_serror_rate  dst_host_srv_serror_rate  dst_host_rerror_rate  \
count         145585.000000             145585.000000         145585.000000   
mean               0.291599                  0.290980              0.110464   
std                0.452493                  0.453448              0.306098   
min                0.000000                  0.000000              0.000000   
25%                0.000000                  0.000000              0.000000   
50%                0.000000                  0.000000              0.000000   
75%                1.000000                  1.000000              0.000000   
max                1.000000                  1.000000              1.000000   

       dst_host_srv_rerror_rate  
count             145585.000000  
mean                   0.107676  
std                    0.304221  
min                    0.00

In [6]:
df = df.sample(frac=1) #shuffle

In [7]:
features = df[['duration', 'src_bytes','dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot','num_failed_logins', 'logged_in', 'lnum_compromised', 'lroot_shell','lsu_attempted', 'lnum_root', 'lnum_file_creations', 'lnum_shells','lnum_access_files', 'lnum_outbound_cmds', 'is_host_login','is_guest_login', 'count', 'srv_count', 'serror_rate','srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate','diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count','dst_host_srv_count', 'dst_host_same_srv_rate','dst_host_diff_srv_rate', 'dst_host_same_src_port_rate','dst_host_srv_diff_host_rate', 'dst_host_serror_rate','dst_host_srv_serror_rate', 'dst_host_rerror_rate','dst_host_srv_rerror_rate']]
label_protocol_type = df['protocol_type']
label_flag = df['flag']
batch_size=64

ds_train = tf.data.Dataset.from_tensor_slices((features[:120000],label_protocol_type[:120000],label_flag[:120000])).batch(batch_size)
ds_test = tf.data.Dataset.from_tensor_slices((features[120000:],label_protocol_type[120000:],label_flag[120000:])).batch(batch_size)

Metal device set to: Apple M1


2022-01-25 21:40:04.416806: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-25 21:40:04.416897: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Creare model

### Predicate

| index | class | 
| --- | --- |
| 0 | TCP |
| 1 | ICMP |
| 2 | UDP |
| 3 | SF |
| 4 | S1 |
| 5 | REJ |
| 6 | S2 |
| 7 | S0 |
| 8 | S3 |
| 9 | RSTO |
| 10 | RSTR |
| 11 | RSTOS0 |
| 12 | OTH |
| 13 | SH |

In [8]:
class MLP(tf.keras.Model):
    """Model that returns logits."""
    def __init__(self, n_classes, hidden_layer_sizes=(16,16,8)):
        super(MLP, self).__init__()
        self.denses = [tf.keras.layers.Dense(s, activation="elu") for s in hidden_layer_sizes]
        self.dense_class = tf.keras.layers.Dense(n_classes)
        
    def call(self, inputs):
        x = inputs
        for dense in self.denses:
            x = dense(x)
        return self.dense_class(x)

logits_model = MLP(14)
p = ltn.Predicate(ltn.utils.LogitsToPredicateModel(logits_model,single_label=False))

In [9]:
class_tcp = ltn.Constant(0, trainable=False)
class_icmp = ltn.Constant(1, trainable=False)
class_udp = ltn.Constant(2, trainable=False)
class_sf = ltn.Constant(3, trainable=False)
class_s1 = ltn.Constant(4, trainable=False)
class_rej = ltn.Constant(5, trainable=False)
class_s2 = ltn.Constant(6, trainable=False)
class_s0 = ltn.Constant(7, trainable=False)
class_s3 = ltn.Constant(8, trainable=False)
class_rsto = ltn.Constant(9, trainable=False)
class_rstr = ltn.Constant(10, trainable=False)
class_rstos0 = ltn.Constant(11, trainable=False)
class_oth = ltn.Constant(12, trainable=False)
class_sh = ltn.Constant(13, trainable=False)

# Logica si axiomele
```
forall x_tcp: C(x_tcp,tcp)
forall x_icmp: C(x_icmp,icmp)
forall x_udp: C(x_udp,udp)
forall x_sf: C(x_sf,sf)
forall x_s1: C(x_s1,s1)
forall x_rej: C(x_rej,rej)
forall x: ~(C(x,tcp) & C(x,sf))
forall x: ~(C(x,icmp) & C(x,s1))
forall x: ~(C(x,udp) & C(x,rej))
```

In [10]:
Not = ltn.Wrapper_Connective(ltn.fuzzy_ops.Not_Std())
And = ltn.Wrapper_Connective(ltn.fuzzy_ops.And_Prod())
Or = ltn.Wrapper_Connective(ltn.fuzzy_ops.Or_ProbSum())
Implies = ltn.Wrapper_Connective(ltn.fuzzy_ops.Implies_Reichenbach())
Equiv = ltn.Wrapper_Connective(ltn.fuzzy_ops.Equiv(ltn.fuzzy_ops.And_Prod(),ltn.fuzzy_ops.Implies_Reichenbach()))
Forall = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMeanError(p=4),semantics="forall")
Exists = ltn.Wrapper_Quantifier(ltn.fuzzy_ops.Aggreg_pMean(p=6),semantics="exists")

In [11]:
formula_aggregator = ltn.Wrapper_Formula_Aggregator(ltn.fuzzy_ops.Aggreg_pMeanError(p=4))

@tf.function
def axioms(features,label_protocol_type,label_flag):
    x = ltn.Variable("x",features)
    x_tcp = ltn.Variable("x_tcp",features[label_protocol_type=="TCP"])
    x_icmp = ltn.Variable("x_icmp",features[label_protocol_type=="ICMP"])
    x_udp = ltn.Variable("x_udp",features[label_protocol_type=="UDP"])
    x_sf = ltn.Variable("x_sf",features[label_flag=="SF"])
    x_s1 = ltn.Variable("x_s1",features[label_flag=="S1"])
    x_rej = ltn.Variable("x_rej",features[label_flag=="REJ"])
    x_s2 = ltn.Variable("x_s2",features[label_flag=="S2"])
    x_s0 = ltn.Variable("x_s0",features[label_flag=="S0"])
    x_s3 = ltn.Variable("x_s3",features[label_flag=="S3"])
    x_rsto = ltn.Variable("x_rsto",features[label_flag=="RSTO"])
    x_rstr = ltn.Variable("x_rstr",features[label_flag=="RSTR"])
    x_rstos0 = ltn.Variable("x_rstos0",features[label_flag=="RSTOS0"])
    x_oth = ltn.Variable("x_oth",features[label_flag=="OTH"])
    x_sh = ltn.Variable("x_sh",features[label_flag=="SH"])
    axioms = [
        Forall(x_tcp, p([x_tcp,class_tcp])),
        Forall(x_udp, p([x_udp,class_udp])),
        Forall(x_icmp, p([x_icmp,class_icmp])),
        Forall(x_sf, p([x_sf,class_sf])),
        Forall(x_s1, p([x_s1,class_s1])),
        Forall(x_rej, p([x_rej,class_rej])),
        # Forall(x_s2, p([x_s2,class_s2])),
        # Forall(x_s0, p([x_s0,class_s0])),
        # Forall(x_s3, p([x_s3,class_s3])),
        # Forall(x_rsto, p([x_rsto,class_rsto])),
        # Forall(x_rstr, p([x_rstr,class_rstr])),
        # Forall(x_rstos0, p([x_rstos0,class_rstos0])),
        # Forall(x_oth, p([x_oth,class_oth])),
        # Forall(x_sh, p([x_sh,class_sh])),
        
        Forall(x,Not(And(p([x,class_tcp]),p([x,class_sf])))),
        Forall(x,Not(And(p([x,class_icmp]),p([x,class_s1])))),
        Forall(x,Not(And(p([x,class_udp]),p([x,class_rej])))),
        
#         Forall(x,Not(And(p([x,class_sf]),p([x,class_s1])))),
#         Forall(x,Not(And(p([x,class_sf]),p([x,class_rej])))),
#         Forall(x,Not(And(p([x,class_sf]),p([x,class_s2])))),
#         Forall(x,Not(And(p([x,class_sf]),p([x,class_s0])))),
#         Forall(x,Not(And(p([x,class_sf]),p([x,class_s3])))),
#         Forall(x,Not(And(p([x,class_sf]),p([x,class_rsto])))),
#         Forall(x,Not(And(p([x,class_sf]),p([x,class_rstr])))),
#         Forall(x,Not(And(p([x,class_sf]),p([x,class_rstos0])))),
#         Forall(x,Not(And(p([x,class_sf]),p([x,class_oth])))),
#         Forall(x,Not(And(p([x,class_sf]),p([x,class_sh])))),
        
#         Forall(x,Not(And(p([x,class_s1]),p([x,class_rej])))),
#         Forall(x,Not(And(p([x,class_s1]),p([x,class_s2])))),
#         Forall(x,Not(And(p([x,class_s1]),p([x,class_s0])))),
#         Forall(x,Not(And(p([x,class_s1]),p([x,class_s3])))),
#         Forall(x,Not(And(p([x,class_s1]),p([x,class_rsto])))),
#         Forall(x,Not(And(p([x,class_s1]),p([x,class_rstr])))),
#         Forall(x,Not(And(p([x,class_s1]),p([x,class_rstos0])))),
#         Forall(x,Not(And(p([x,class_s1]),p([x,class_oth])))),
#         Forall(x,Not(And(p([x,class_s1]),p([x,class_sh])))),
        
#         Forall(x,Not(And(p([x,class_rej]),p([x,class_s2])))),
#         Forall(x,Not(And(p([x,class_rej]),p([x,class_s0])))),
#         Forall(x,Not(And(p([x,class_rej]),p([x,class_s3])))),
#         Forall(x,Not(And(p([x,class_rej]),p([x,class_rsto])))),
#         Forall(x,Not(And(p([x,class_rej]),p([x,class_rstr])))),
#         Forall(x,Not(And(p([x,class_rej]),p([x,class_rstos0])))),
#         Forall(x,Not(And(p([x,class_rej]),p([x,class_oth])))),
#         Forall(x,Not(And(p([x,class_rej]),p([x,class_sh])))),
        
#         Forall(x,Not(And(p([x,class_s2]),p([x,class_s0])))),
#         Forall(x,Not(And(p([x,class_s2]),p([x,class_s3])))),
#         Forall(x,Not(And(p([x,class_s2]),p([x,class_rsto])))),
#         Forall(x,Not(And(p([x,class_s2]),p([x,class_rstr])))),
#         Forall(x,Not(And(p([x,class_s2]),p([x,class_rstos0])))),
#         Forall(x,Not(And(p([x,class_s2]),p([x,class_oth])))),
#         Forall(x,Not(And(p([x,class_s2]),p([x,class_sh])))),
        
#         Forall(x,Not(And(p([x,class_s0]),p([x,class_s3])))),
#         Forall(x,Not(And(p([x,class_s0]),p([x,class_rsto])))),
#         Forall(x,Not(And(p([x,class_s0]),p([x,class_rstr])))),
#         Forall(x,Not(And(p([x,class_s0]),p([x,class_rstos0])))),
#         Forall(x,Not(And(p([x,class_s0]),p([x,class_oth])))),
#         Forall(x,Not(And(p([x,class_s0]),p([x,class_sh])))),
        
#         Forall(x,Not(And(p([x,class_s3]),p([x,class_rsto])))),
#         Forall(x,Not(And(p([x,class_s3]),p([x,class_rstr])))),
#         Forall(x,Not(And(p([x,class_s3]),p([x,class_rstos0])))),
#         Forall(x,Not(And(p([x,class_s3]),p([x,class_oth])))),
#         Forall(x,Not(And(p([x,class_s3]),p([x,class_sh])))),
        
#         Forall(x,Not(And(p([x,class_rsto]),p([x,class_rstr])))),
#         Forall(x,Not(And(p([x,class_rsto]),p([x,class_rstos0])))),
#         Forall(x,Not(And(p([x,class_rsto]),p([x,class_oth])))),
#         Forall(x,Not(And(p([x,class_rsto]),p([x,class_sh])))),
        
#         Forall(x,Not(And(p([x,class_rstr]),p([x,class_rstos0])))),
#         Forall(x,Not(And(p([x,class_rstr]),p([x,class_oth])))),
#         Forall(x,Not(And(p([x,class_rstr]),p([x,class_sh])))),
        
#         Forall(x,Not(And(p([x,class_rstos0]),p([x,class_oth])))),
#         Forall(x,Not(And(p([x,class_rstos0]),p([x,class_sh])))),
        
#         Forall(x,Not(And(p([x,class_oth]),p([x,class_sh])))),
    ]
    sat_level = formula_aggregator(axioms).tensor
    return sat_level

In [12]:
for features, label_protocol_type, label_flag in ds_train:
    print("Initial sat level %.5f"%axioms(features,label_protocol_type,label_flag))
    break

2022-01-25 21:40:06.198227: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-25 21:40:06.198551: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Initial sat level 0.33618


# Training

In [13]:
metrics_dict = {
    'train_sat_kb': tf.keras.metrics.Mean(name='train_sat_kb'),
    'test_sat_kb': tf.keras.metrics.Mean(name='test_sat_kb'),
    'train_accuracy': tf.keras.metrics.Mean(name="train_accuracy"),
    'test_accuracy': tf.keras.metrics.Mean(name="test_accuracy"),
    'test_sat_phi1': tf.keras.metrics.Mean(name='test_sat_phi1'),
    'test_sat_phi2': tf.keras.metrics.Mean(name='test_sat_phi2'),
    'test_sat_phi3': tf.keras.metrics.Mean(name='test_sat_phi3')
}

@tf.function()
def sat_phi1(features):
    x = ltn.Variable("x",features)
    phi1 = Forall(x, Implies(p([x,class_udp]),Not(p([x,class_tcp]))),p=5)
    return phi1.tensor

@tf.function()
def sat_phi2(features):
    x = ltn.Variable("x",features)
    phi2 = Forall(x, Implies(p([x,class_udp]),p([x,class_tcp])),p=5)
    return phi2.tensor

@tf.function()
def sat_phi3(features):
    x = ltn.Variable("x",features)
    phi3 = Forall(x, Implies(p([x,class_tcp]),p([x,class_sf])),p=5)
    return phi3.tensor

def multilabel_hamming_loss(y_true, y_pred, threshold=0.5,from_logits=False):
    if from_logits:
        y_pred = tf.math.sigmoid(y_pred)
    y_pred = y_pred > threshold
    y_true = tf.cast(y_true, tf.int32)
    y_pred = tf.cast(y_pred, tf.int32)
    nonzero = tf.cast(tf.math.count_nonzero(y_true-y_pred,axis=-1),tf.float32)
    return nonzero/y_true.get_shape()[-1]

In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
@tf.function
def train_step(features, label_protocol_type, label_flag):
    # sat and update
    with tf.GradientTape() as tape:
        sat = axioms(features, label_protocol_type, label_flag)
        loss = 1.-sat
    gradients = tape.gradient(loss, p.trainable_variables)
    optimizer.apply_gradients(zip(gradients, p.trainable_variables))
    metrics_dict['train_sat_kb'](sat)
    # accuracy
    predictions = logits_model(features)
    label_tcp = (label_protocol_type == "TCP")
    label_icmp = (label_protocol_type == "ICMP")
    label_udp = (label_protocol_type == "UDP")
    label_sf = (label_flag == "SF")
    label_s1 = (label_flag == "S1")
    label_rej = (label_flag == "REJ")
    label_s2 = (label_flag == "S2")
    label_s0 = (label_flag == "S0")
    label_s3 = (label_flag == "S3")
    label_rsto = (label_flag == "RSTO")
    label_rstr = (label_flag == "RSTR")
    label_rstos0 = (label_flag == "RSTOS0")
    label_oth = (label_flag == "OTH")
    label_sh = (label_flag == "SH")
    

    onehot = tf.stack([label_tcp,label_icmp,label_udp,label_sf,label_s1,label_rej,label_s2,label_s0,label_s3,label_rsto,label_rstr,label_rstos0,label_oth,label_sh],axis=-1)
    #onehot = tf.stack([label_tcp,label_icmp,label_udp,label_sf],axis=-1)
    metrics_dict['train_accuracy'](1-multilabel_hamming_loss(onehot,predictions,from_logits=True))
    
@tf.function
def test_step(features, label_protocol_type, label_flag):
    # sat
    sat_kb = axioms(features, label_protocol_type, label_flag)
    metrics_dict['test_sat_kb'](sat_kb)
    metrics_dict['test_sat_phi1'](sat_phi1(features))
    metrics_dict['test_sat_phi2'](sat_phi2(features))
    metrics_dict['test_sat_phi3'](sat_phi3(features))
    # accuracy
    predictions = logits_model(features)
    label_tcp = (label_protocol_type == "TCP")
    label_icmp = (label_protocol_type == "ICMP")
    label_udp = (label_protocol_type == "UDP")
    label_sf = (label_flag == "SF")
    label_s1 = (label_flag == "S1")
    label_rej = (label_flag == "REJ")
    label_s2 = (label_flag == "S2")
    label_s0 = (label_flag == "S0")
    label_s3 = (label_flag == "S3")
    label_rsto = (label_flag == "RSTO")
    label_rstr = (label_flag == "RSTR")
    label_rstos0 = (label_flag == "RSTOS0")
    label_oth = (label_flag == "OTH")
    label_sh = (label_flag == "SH")
    onehot = tf.stack([label_tcp,label_icmp,label_udp,label_sf,label_s1,label_rej,label_s2,label_s0,label_s3,label_rsto,label_rstr,label_rstos0,label_oth,label_sh],axis=-1)
    #onehot = tf.stack([label_tcp,label_icmp,label_udp,label_sf],axis=-1)
    metrics_dict['test_accuracy'](1-multilabel_hamming_loss(onehot,predictions,from_logits=True))

In [15]:
from collections import defaultdict

def train(
        epochs,
        metrics_dict, 
        ds_train, 
        ds_test, 
        train_step, 
        test_step,
        track_metrics=1,
        csv_path=None,
        scheduled_parameters=defaultdict(lambda : {})
    ):
    """
    Args:
        epochs: int, number of training epochs.
        metrics_dict: dict, {"metrics_label": tf.keras.metrics instance}.
        ds_train: iterable dataset, e.g. using tf.data.Dataset.
        ds_test: iterable dataset, e.g. using tf.data.Dataset.
        train_step: callable function. the arguments passed to the function
            are the itered elements of ds_train.
        test_step: callable function. the arguments passed to the function
            are the itered elements of ds_test.
        csv_path: (optional) path to create a csv file, to save the metrics.
        scheduled_parameters: (optional) a dictionary that returns kwargs for
            the train_step and test_step functions, for each epoch.
            Call using scheduled_parameters[epoch].
    """
    template = "Epoch {}"
    for metrics_label in metrics_dict.keys():
        template += ", %s: {:.4f}" % metrics_label
    if csv_path is not None:
        csv_file = open(csv_path,"w+")
        headers = ",".join(["Epoch"]+list(metrics_dict.keys()))
        csv_template = ",".join(["{}" for _ in range(len(metrics_dict)+1)])
        csv_file.write(headers+"\n")
    
    for epoch in range(epochs):
        for metrics in metrics_dict.values():
            metrics.reset_states()

        for batch_elements in ds_train:
            train_step(*batch_elements,**scheduled_parameters[epoch])
        for batch_elements in ds_test:
            test_step(*batch_elements,**scheduled_parameters[epoch])

        metrics_results = [metrics.result() for metrics in metrics_dict.values()]
        if epoch%track_metrics == 0:
            print(template.format(epoch,*metrics_results))
        if csv_path is not None:
            csv_file.write(csv_template.format(epoch,*metrics_results)+"\n")
            csv_file.flush()
    if csv_path is not None:
        csv_file.close()

In [16]:
EPOCHS = 20
train(
    EPOCHS,
    metrics_dict,
    ds_train,
    ds_test,
    train_step,
    test_step,
    csv_path="results.csv",
    track_metrics=20
)

end_time = time.time()


2022-01-25 21:40:07.122021: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-25 21:41:34.089791: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-25 21:41:45.302566: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 0, train_sat_kb: 0.8647, test_sat_kb: 0.9738, train_accuracy: 0.4954, test_accuracy: 0.5004, test_sat_phi1: 0.9999, test_sat_phi2: 0.9998, test_sat_phi3: 0.9999


In [17]:
print("It took", end =" ")
total_time = round(end_time-start_time,3)
print(total_time , end ="")
print("s to run")

It took 2023.613s to run
